In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 46.9 MB/s eta 0:00:00


In [3]:
!pip install duckdb

In [4]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.9 MB/s eta 0:00:00


In [5]:
!pip install xgboost

In [16]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
import lightgbm as lgb
import numpy as np
import optuna

In [7]:
import duckdb
import pandas as pd

train_path = '/content/drive/MyDrive/BELKA-DrugPredict/data/train.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 50000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 50000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,287670690,O=C(O)[C@H]1CC2CCCCC2N1C(=O)OCC1c2ccccc2-c2ccc...,CC(C)c1nnc([C@H]2C[C@H](CN)[C@H](O)C2)[nH]1,Cc1cc(Cl)nc(Cl)c1N,Cc1cc(Cl)nc(Cl)c1Nc1nc(NC[C@H]2C[C@H](c3nnc(C(...,BRD4,0
1,171991644,O=C(Nc1cc(Cl)ncc1C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1ccc([N+](=O)[O-])cn1,Cl.NCCC1CC2(CCC2)CO1,O=C(N[Dy])c1cnc(Cl)cc1Nc1nc(NCCC2CC3(CCC3)CO2)...,BRD4,0
2,46805904,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Nc1nc(NCc2ccc(F)cc2)ccc1[N+](=O)[O-],NCc1ccc(Oc2cccnc2)c(F)c1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(Oc3cccn...,BRD4,0
3,16113811,CC(C)(C)OC(=O)N1C[C@@H](NC(=O)OCC2c3ccccc3-c3c...,Nc1cc(Cl)ncn1,CC1(C)NC(=O)N(CCCN)C1=O.Cl,CC(C)(C)OC(=O)N1C[C@@H](Nc2nc(NCCCN3C(=O)NC(C)...,HSA,0
4,80635346,O=C(NC1(C(=O)O)CCc2ccccc21)OCC1c2ccccc2-c2ccccc21,C=CCOCCCN,NCc1cccc(C(=O)N2CCCC2)c1,C=CCOCCCNc1nc(NCc2cccc(C(=O)N3CCCC3)c2)nc(NC2(...,sEH,0


In [9]:
# ECFP 생성 함수
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

# 분자의 물리화학적 특성을 추출하는 함수
def generate_physchem_features(molecule):
    if molecule is None:
        return [np.nan] * 8
    return [
        Descriptors.MolWt(molecule),
        Descriptors.MolLogP(molecule),
        Descriptors.NumHDonors(molecule),
        Descriptors.NumHAcceptors(molecule),
        Descriptors.TPSA(molecule),
        Descriptors.NumRotatableBonds(molecule),
        Descriptors.RingCount(molecule),
        Descriptors.BalabanJ(molecule)
    ]

In [10]:
# 데이터 전처리
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)
df['ecfp'] = df['molecule'].apply(generate_ecfp)
df['physchem'] = df['molecule'].apply(generate_physchem_features)

In [11]:
# ECFP와 물리화학적 특성을 결합
df['features'] = df.apply(lambda row: row['ecfp'] + row['physchem'], axis=1)

In [12]:
# 단백질 이름을 원-핫 인코딩
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

## Train

In [13]:
# 최종 입력 데이터 생성
X = [features + list(protein) for features, protein in zip(df['features'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

In [14]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Optuna를 사용하여 하이퍼파라미터 최적화
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0)
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    map_score = average_precision_score(y_test, y_pred_proba)
    return map_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-05-29 06:31:50,324] A new study created in memory with name: no-name-14da65de-1a97-4de0-8590-a27ed77240ab
<ipython-input-17-a4b526731fcb>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
<ipython-input-17-a4b526731fcb>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
<ipython-input-17-a4b526731fcb>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6,

[LightGBM] [Info] Number of positive: 40035, number of negative: 39965
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.109459 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3089
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 1016
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500437 -> initscore=0.001750
[LightGBM] [Info] Start training from score 0.001750
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-05-29 06:31:59,438] Trial 0 finished with value: 0.9445854384248791 and parameters: {'n_estimators': 115, 'max_depth': 3, 'learning_rate': 0.21660108843731957, 'subsample': 0.710954945700892, 'colsample_bytree': 0.6960155004595843}. Best is trial 0 with value: 0.9445854384248791.
<ipython-input-17-a4b526731fcb>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
<ipython-input-17-a4b526731fcb>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
<ipython-input-17-a4b526731fcb>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will 

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 40035, number of negative: 39965
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.110338 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3089
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 1016
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500437 -> initscore=0.001750
[LightGBM] [Info] Start training from score 0.001750


[I 2024-05-29 06:32:08,933] Trial 1 finished with value: 0.9308344580411898 and parameters: {'n_estimators': 130, 'max_depth': 10, 'learning_rate': 0.013041679019118, 'subsample': 0.8986109135692614, 'colsample_bytree': 0.6763814363287616}. Best is trial 0 with value: 0.9445854384248791.


...

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


<ipython-input-17-a4b526731fcb>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
<ipython-input-17-a4b526731fcb>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
<ipython-input-17-a4b526731fcb>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 40035, number of negative: 39965
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.111129 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3089
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 1016
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500437 -> initscore=0.001750
[LightGBM] [Info] Start training from score 0.001750


[I 2024-05-29 06:40:00,389] Trial 49 finished with value: 0.9661287526120776 and parameters: {'n_estimators': 87, 'max_depth': 10, 'learning_rate': 0.24666343959664075, 'subsample': 0.8133160134691382, 'colsample_bytree': 0.9536545796704361}. Best is trial 38 with value: 0.9715442842912791.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [18]:
# 최적의 하이퍼파라미터로 모델 생성
best_params = study.best_params
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_train, y_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 40035, number of negative: 39965
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.111636 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3089
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 1016
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500437 -> initscore=0.001750
[LightGBM] [Info] Start training from score 0.001750


LGBMClassifier(colsample_bytree=0.9660008787948949,
               learning_rate=0.24216874536217714, max_depth=10,
               n_estimators=244, subsample=0.8081376619799521)

## Submission

In [19]:
# 테스트 데이터 예측 및 저장
test_file = '/content/drive/MyDrive/BELKA-DrugPredict/data/test.csv'
output_file = '/content/drive/MyDrive/BELKA-DrugPredict/output/submission2.csv'

In [20]:
import os

In [21]:
for df_test in pd.read_csv(test_file, chunksize=100000):
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)
    df_test['physchem'] = df_test['molecule'].apply(generate_physchem_features)
    df_test['features'] = df_test.apply(lambda row: row['ecfp'] + row['physchem'], axis=1)
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))
    X_test = [features + list(protein) for features, protein in zip(df_test['features'].tolist(), protein_onehot.tolist())]
    probabilities = best_model.predict_proba(X_test)[:, 1]
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^m